In [0]:
import sys 
!{sys.executable} -m pip install tensorflow

In [0]:
import sys 
!{sys.executable} -m pip install opencv-python


In [0]:
import os
import numpy as np
import cv2
import keras.backend as K
import random
import tensorflow as tf


def load_image(Path = './valid/XR_SHOULDER', size = 512):
    Images = []
    for path in Path:
        try:
            image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            Images.append(image)

        except Exception as e:
            print(str(e))

    Images = np.asarray(Images).astype('float32')
    Images=Images.reshape((len(Path),size,size,1))

    mean = np.mean(Images)			#normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    return Images

Using TensorFlow backend.


In [0]:
def load_path(root_path = './valid/XR_SHOULDER', size = 512):
	'''
	load MURA dataset
	'''

	Path = []
	labels = []
	for root,dirs,files in os.walk(root_path): #Read all pictures, os.walk Return to iterator genertor Traverse all files
		for name in files:
			path_1 = os.path.join(root,name)
			Path.append(path_1)
			if root.split('_')[-1]=='positive':	 #positive Label is 1，otherwise 0；
				labels+=[1]   	          	 #Last level directory file patient11880\\study1_negative\\image3.png
			else:
			    labels+=[0]
	print (len(Path))
	labels = np.asarray(labels)
	return Path, labels


Path,labels=load_path(r'/content/MURA-v1.1/train/XR_SHOULDER',64)
images=load_image(Path,64)

8379


In [0]:
     def train_model():
        model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu,input_shape=(64,64,1)),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),       
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(32,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(2,activation=tf.nn.softmax)
        ])
        model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
        model.fit(images,labels,epochs=40,verbose=2)
     
        return model

In [0]:
model=train_model()

Epoch 1/40
 - 9s - loss: 0.6918 - acc: 0.5435
Epoch 2/40
 - 8s - loss: 0.6613 - acc: 0.6071
Epoch 3/40
 - 8s - loss: 0.6350 - acc: 0.6432
Epoch 4/40
 - 9s - loss: 0.6093 - acc: 0.6674
Epoch 5/40
 - 9s - loss: 0.5958 - acc: 0.6778
Epoch 6/40
 - 8s - loss: 0.5725 - acc: 0.7031
Epoch 7/40
 - 9s - loss: 0.5499 - acc: 0.7203
Epoch 8/40
 - 8s - loss: 0.5298 - acc: 0.7336
Epoch 9/40
 - 8s - loss: 0.5071 - acc: 0.7508
Epoch 10/40
 - 8s - loss: 0.4813 - acc: 0.7667
Epoch 11/40
 - 8s - loss: 0.4619 - acc: 0.7735
Epoch 12/40
 - 9s - loss: 0.4381 - acc: 0.7966
Epoch 13/40
 - 9s - loss: 0.4192 - acc: 0.8020
Epoch 14/40
 - 9s - loss: 0.3840 - acc: 0.8259
Epoch 15/40
 - 8s - loss: 0.3612 - acc: 0.8369
Epoch 16/40
 - 9s - loss: 0.3405 - acc: 0.8505
Epoch 17/40
 - 9s - loss: 0.3184 - acc: 0.8588
Epoch 18/40
 - 10s - loss: 0.2984 - acc: 0.8715
Epoch 19/40
 - 10s - loss: 0.2799 - acc: 0.8801
Epoch 20/40
 - 9s - loss: 0.2583 - acc: 0.8885
Epoch 21/40
 - 9s - loss: 0.2383 - acc: 0.8981
Epoch 22/40
 - 9s - 

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               401536    
__________

In [0]:
#Validation of test data
ValidPath,Validlabels=load_path(r'/content/MURA-v1.1/valid/XR_SHOULDER',64)
Validimages=load_image(ValidPath,64)

563


In [0]:
score = model.evaluate(Validimages, Validlabels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.4193648698063135
Test accuracy: 0.660746


In [0]:
model.save('shoulder.h5')